In [36]:
from PIL import Image
import cv2 
import numpy as np
from IPython.display import display

In [37]:
import glob, os

laser_samples = []
os.chdir("../res/laser_samples/")
for file in glob.glob("*"):
    laser_samples.append(file)

FileNotFoundError: [Errno 2] No such file or directory: '../res/laser_samples/'

In [ ]:
def disp_file(path):
    # In OpenCV, images are in BGR
    # In Pillow, images are in RGB 
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    display(Image.fromarray(img))


# Display cv2 image
def disp_img(cv_img):
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    display(Image.fromarray(cv_img))


In [ ]:
for file in laser_samples:
    disp_file(path=file)

## Idea 1. HSV Isolation
Manually tune the lower and upper bounds of HSV values to obtain laser line

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, HBox

In [ ]:
@interact
def hsv_isolation(lh=(0,255), ls=(0,255), lv=(0,255), uh=(0,255), us=(0,255), uv=(0,255)):
    img = cv2.imread(laser_samples[1])

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # define range of blue color in HSV
    lower_colour = np.array([lh, ls, lv])
    upper_colour = np.array([uh, us, uv])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_colour, upper_colour)
    # inv = cv2.bitwise_not(mask)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(img,img, mask= mask)

    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)

    display(Image.fromarray(res))


In [ ]:
@interact
def hsv_isolation(lh=(0,255), ls=(0,255), lv=(0,255), uh=(0,255), us=(0,255), uv=(0,255)):
    img = cv2.imread(laser_samples[0])

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # define range of blue color in HSV
    lower_colour = np.array([lh, ls, lv])
    upper_colour = np.array([uh, us, uv])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_colour, upper_colour)
    # inv = cv2.bitwise_not(mask)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(img,img, mask= mask)

    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)

    display(Image.fromarray(res))

## Idea 2. Hough Transform
Used to extract lines

## Idea 3. Gradients

## Idea 4. Dilation and Erosion